In [4]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split

import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torchtext.utils import download_from_url, extract_archive
import io
from nltk.tokenize import word_tokenize
import re

from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import optim

In [5]:
# todos 1) Make generalize tokenizer 2) make preprocessing different so we can use while custom dataset
class DateDataset(Dataset):
    def __init__(self, DATASET_PATH = "dataset/Assignment2aDataset.txt", split="train"):
        df = pd.read_csv(DATASET_PATH, names = ["source", "target"])
        df["source"] = df["source"].apply(lambda x: x.strip()[1:-1].replace("/", "-"))
        df["target"] = df["target"].apply(lambda x: x.strip()[1:-1])
        df_train, df_test = train_test_split(df, random_state=42, test_size=0.1)
        
        # tokenize
        en_tokenizer = get_tokenizer('spacy', language='en')   
        counter = Counter() # dict of {token: Freq}     
        for source in df["source"]:
            counter.update(en_tokenizer(source))

        for source in df["target"]:
            counter.update(en_tokenizer(source))
        
        voc = vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])    
        
        # create data
        if split == "train":
            self.data_df = df_train
        else:
            self.data_df = df_test
            
        data = []
        for (source, target) in zip(self.data_df["source"], self.data_df["target"]):
            s_tensor_ = torch.tensor([voc[token] for token in en_tokenizer(source)])
            t_tensor_ = torch.tensor([voc[token] for token in en_tokenizer(target)])
            data.append((s_tensor_, t_tensor_))
        
        self.voc = voc
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    


/home/karm/miniconda3/envs/torch310/lib/python3.11/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [6]:
BOS_IDX = train_dataset.voc["<bos>"]
EOS_IDX = train_dataset.voc["<eos>"]
PAD_IDX = train_dataset.voc["<pad>"]

def generate_batch(data_batch):
    s_batch, t_batch = [], []
    for (s_item, t_item) in data_batch:
        s_batch.append(torch.cat([torch.tensor([BOS_IDX]), s_item, torch.tensor([EOS_IDX])], dim=0))
        t_batch.append(torch.cat([torch.tensor([BOS_IDX]), t_item, torch.tensor([EOS_IDX])], dim=0))
        
    s_batch = pad_sequence(s_batch, padding_value=PAD_IDX)
    return s_batch.T, torch.stack(t_batch)



In [8]:
train_dataset = DateDataset(split="train")
train_dataloader = DataLoader(train_dataset, batch_size=64, collate_fn=generate_batch)

/home/karm/miniconda3/envs/torch310/lib/python3.11/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [9]:
test_dataset = DateDataset(split="test")
test_dataloader = DataLoader(test_dataset, batch_size=64, collate_fn=generate_batch)

/home/karm/miniconda3/envs/torch310/lib/python3.11/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
